In [1]:
import numpy as np
import pandas as pd


In [2]:
def read_epw(filename):
    # Skip the header lines
    header_lines = 8  # Number of lines in the header
    data = pd.read_csv(filename, skiprows=header_lines, header=None)

    # Assign column names based on the IDD description
    column_names = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Data Source and Uncertainty Flags',
                    'Dry Bulb Temperature', 'Dew Point Temperature', 'Relative Humidity',
                    'Atmospheric Station Pressure', 'Extraterrestrial Horizontal Radiation',
                    'Extraterrestrial Direct Normal Radiation', 'Horizontal Infrared Radiation Intensity',
                    'Global Horizontal Radiation', 'Direct Normal Radiation', 'Diffuse Horizontal Radiation',
                    'Global Horizontal Illuminance', 'Direct Normal Illuminance', 'Diffuse Horizontal Illuminance',
                    'Zenith Luminance', 'Wind Direction', 'Wind Speed', 'Total Sky Cover',
                    'Opaque Sky Cover (used if Horizontal IR Intensity missing)', 'Visibility',
                    'Ceiling Height', 'Present Weather Observation', 'Present Weather Codes',
                    'Precipitable Water', 'Aerosol Optical Depth', 'Snow Depth', 'Days Since Last Snowfall',
                    'Albedo', 'Liquid Precipitation Depth', 'Liquid Precipitation Quantity']

    # Assign column names to DataFrame
    data.columns = column_names
    
    # Convert 'Year', 'Month', 'Day', 'Hour', 'Minute' columns to datetime
    data['datetime'] = pd.to_datetime(data[['Year', 'Month', 'Day', 'Hour', 'Minute']])
    
    # Drop the original date and time columns
    data = data.drop(columns=['Year', 'Minute', 'Data Source and Uncertainty Flags',
                    
                     'Extraterrestrial Horizontal Radiation',
                    'Extraterrestrial Direct Normal Radiation', 'Horizontal Infrared Radiation Intensity',
                    
                    'Zenith Luminance', 'Opaque Sky Cover (used if Horizontal IR Intensity missing)',
                    'Visibility',
                    'Ceiling Height', 'Present Weather Observation', 'Present Weather Codes',
                    'Aerosol Optical Depth', 'Snow Depth', 'Days Since Last Snowfall',
                    'Albedo'])
    
    return data

In [3]:
file='C:/Users/Amresh Gupta/OneDrive/Desktop/Research Project PCM/1000 wather files/1 to 1157 corrected/508-hour.epw'
df = read_epw(file)


In [4]:
df.columns

Index(['Month', 'Day', 'Hour', 'Dry Bulb Temperature', 'Dew Point Temperature',
       'Relative Humidity', 'Atmospheric Station Pressure',
       'Global Horizontal Radiation', 'Direct Normal Radiation',
       'Diffuse Horizontal Radiation', 'Global Horizontal Illuminance',
       'Direct Normal Illuminance', 'Diffuse Horizontal Illuminance',
       'Wind Direction', 'Wind Speed', 'Total Sky Cover', 'Precipitable Water',
       'Liquid Precipitation Depth', 'Liquid Precipitation Quantity',
       'datetime'],
      dtype='object')

In [5]:
df['latitude'] = [22.98] * 8760
df['longitude'] = [75.61] * 8760

In [6]:
df

,Month,Day,Hour,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,...,Diffuse Horizontal Illuminance,Wind Direction,Wind Speed,Total Sky Cover,Precipitable Water,Liquid Precipitation Depth,Liquid Precipitation Quantity,datetime,latitude,longitude
0,1,1,1,17.3,10.6,65,95757,0,0,0,...,0,83,1.3,6,20,0.0,99.0,2005-01-01 02:00:00,22.98,75.61
1,1,1,2,17.1,10.4,65,95757,0,0,0,...,0,162,1.0,6,19,0.0,99.0,2005-01-01 03:00:00,22.98,75.61
2,1,1,3,16.9,9.8,63,95757,0,0,0,...,0,249,1.0,6,18,0.0,99.0,2005-01-01 04:00:00,22.98,75.61
3,1,1,4,16.9,10.7,67,95757,0,0,0,...,0,310,1.4,6,20,0.0,99.0,2005-01-01 05:00:00,22.98,75.61
4,1,1,5,16.8,10.6,67,95757,0,0,0,...,0,218,0.9,6,19,0.0,99.0,2005-01-01 06:00:00,22.98,75.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12,31,20,20.2,7.0,42,95801,0,0,0,...,0,129,1.6,3,15,0.0,99.0,2005-12-31 21:00:00,22.98,75.61
8756,12,31,21,18.4,7.3,49,95795,0,0,0,...,0,24,0.8,3,16,0.0,99.0,2005-12-31 22:00:00,22.98,75.61
8757,12,31,22,16.5,7.0,53,95788,0,0,0,...,0,165,0.1,3,15,0.0,99.0,2005-12-31 23:00:00,22.98,75.61
8758,12,31,23,14.6,6.8,59,95782,0,0,0,...,0,73,0.1,3,15,0.0,99.0,2006-01-01 00:00:00,22.98,75.61


In [7]:
df['hour'] = (df['Hour'] - 1) % 24
df['datetime'] = pd.to_datetime(df[['Month', 'Day', 'hour']].astype(str).agg('-'.join, axis=1) + '-2005', format='%m-%d-%H-%Y')
df['Days'] = df['datetime'].dt.date
df['min_DBtemp_per_day'] = df.groupby(['Days', 'latitude', 'longitude'])['Dry Bulb Temperature'].transform('min')
df['max_DBT_per_day'] = df.groupby(['Days', 'latitude', 'longitude'])['Dry Bulb Temperature'].transform('max')
df=df.drop(columns='hour',axis=1)

In [8]:
df['max_DNR_per_day'] = df.groupby(['Days', 'latitude', 'longitude'])['Direct Normal Radiation'].transform('max')
df['max_GHI_per_day'] = df.groupby(['Days', 'latitude', 'longitude'])['Global Horizontal Radiation'].transform('max')


In [9]:
df['isday'] = df['Hour'].apply(lambda x: 1 if x in (5, 19) else 0)

In [10]:
df['mean_RH_per_day'] = df.groupby(['Days', 'latitude', 'longitude'])['Relative Humidity'].transform('mean')
df['mean_GHI_daytime'] = df.groupby(['Days', 'latitude', 'longitude','isday'])['Global Horizontal Radiation'].transform('mean')
df['mean_DNR_daytime'] = df.groupby(['Days', 'latitude', 'longitude','isday'])['Direct Normal Radiation'].transform('mean')
df['mean_TSC_daytime'] = df.groupby(['Days', 'latitude', 'longitude','isday'])['Total Sky Cover'].transform('mean')
df['mean_WS'] = df.groupby(['Days', 'latitude', 'longitude'])['Wind Speed'].transform('mean')
df['mean_WD'] = df.groupby(['Days', 'latitude', 'longitude'])['Wind Direction'].transform('mean')
df['mean_DHR_daytime'] = df.groupby(['Days', 'latitude', 'longitude','isday'])['Diffuse Horizontal Radiation'].transform('mean')


In [11]:
df.columns

Index(['Month', 'Day', 'Hour', 'Dry Bulb Temperature', 'Dew Point Temperature',
       'Relative Humidity', 'Atmospheric Station Pressure',
       'Global Horizontal Radiation', 'Direct Normal Radiation',
       'Diffuse Horizontal Radiation', 'Global Horizontal Illuminance',
       'Direct Normal Illuminance', 'Diffuse Horizontal Illuminance',
       'Wind Direction', 'Wind Speed', 'Total Sky Cover', 'Precipitable Water',
       'Liquid Precipitation Depth', 'Liquid Precipitation Quantity',
       'datetime', 'latitude', 'longitude', 'Days', 'min_DBtemp_per_day',
       'max_DBT_per_day', 'max_DNR_per_day', 'max_GHI_per_day', 'isday',
       'mean_RH_per_day', 'mean_GHI_daytime', 'mean_DNR_daytime',
       'mean_TSC_daytime', 'mean_WS', 'mean_WD', 'mean_DHR_daytime'],
      dtype='object')

In [12]:
df.iloc[40:80,:]

,Month,Day,Hour,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,...,max_DNR_per_day,max_GHI_per_day,isday,mean_RH_per_day,mean_GHI_daytime,mean_DNR_daytime,mean_TSC_daytime,mean_WS,mean_WD,mean_DHR_daytime
40,1,2,17,24.2,7.7,35,95799,233,452,109,...,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
41,1,2,18,22.4,7.3,38,95805,22,0,22,...,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
42,1,2,19,21.3,9.1,45,95810,0,0,0,...,845,697,1,53.083333,0.000000,0.000000,4.500000,2.162500,154.458333,0.000000
43,1,2,20,20.3,9.1,49,95816,0,0,0,...,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
44,1,2,21,19.2,8.8,51,95821,0,0,0,...,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
45,1,2,22,18.1,9.0,55,95827,0,0,0,...,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
46,1,2,23,17.0,9.1,59,95832,0,0,0,...,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
47,1,2,24,15.9,8.6,62,95837,0,0,0,...,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
48,1,3,1,15.4,8.8,65,95843,0,0,0,...,1005,774,0,45.791667,224.454545,347.681818,2.727273,1.925000,244.958333,33.363636
49,1,3,2,14.8,9.5,70,95848,0,0,0,...,1005,774,0,45.791667,224.454545,347.681818,2.727273,1.925000,244.958333,33.363636


In [13]:
nndf=df.drop(columns=['Hour', 'Dry Bulb Temperature', 'Dew Point Temperature',
       'Relative Humidity', 'Atmospheric Station Pressure',
       'Global Horizontal Radiation', 'Direct Normal Radiation',
       'Diffuse Horizontal Radiation', 'Global Horizontal Illuminance',
       'Direct Normal Illuminance', 'Diffuse Horizontal Illuminance',
       'Wind Direction', 'Wind Speed', 'Total Sky Cover', 'Precipitable Water',
       'Liquid Precipitation Depth', 'Liquid Precipitation Quantity',
       'datetime',],axis=1)

In [14]:
nndf=nndf.drop_duplicates().reset_index(drop=True)

In [15]:
nndf.shape


(730, 17)

In [16]:
nndf

,Month,Day,latitude,longitude,Days,min_DBtemp_per_day,max_DBT_per_day,max_DNR_per_day,max_GHI_per_day,isday,mean_RH_per_day,mean_GHI_daytime,mean_DNR_daytime,mean_TSC_daytime,mean_WS,mean_WD,mean_DHR_daytime
0,1,1,22.98,75.61,2005-01-01,16.7,31.2,756,663,0,46.875000,192.181818,242.227273,4.136364,2.329167,133.875000,58.818182
1,1,1,22.98,75.61,2005-01-01,16.7,31.2,756,663,1,46.875000,0.000000,0.000000,6.000000,2.329167,133.875000,0.000000
2,1,2,22.98,75.61,2005-01-02,14.3,25.8,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
3,1,2,22.98,75.61,2005-01-02,14.3,25.8,845,697,1,53.083333,0.000000,0.000000,4.500000,2.162500,154.458333,0.000000
4,1,3,22.98,75.61,2005-01-03,13.3,29.0,1005,774,0,45.791667,224.454545,347.681818,2.727273,1.925000,244.958333,33.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,12,29,22.98,75.61,2005-12-29,13.7,28.0,773,682,1,47.416667,0.000000,0.000000,4.000000,3.508333,200.125000,0.000000
726,12,30,22.98,75.61,2005-12-30,14.9,28.5,773,683,0,47.666667,197.181818,268.545455,2.363636,2.204167,193.708333,52.090909
727,12,30,22.98,75.61,2005-12-30,14.9,28.5,773,683,1,47.666667,0.000000,0.000000,3.500000,2.204167,193.708333,0.000000
728,12,31,22.98,75.61,2005-12-31,12.8,27.6,935,735,0,46.833333,212.909091,324.818182,2.045455,1.995833,106.250000,35.590909


In [17]:
nndf = nndf[nndf['isday'] != 1]

In [18]:
nndf=nndf.reset_index(drop=True)

In [19]:
nndf

,Month,Day,latitude,longitude,Days,min_DBtemp_per_day,max_DBT_per_day,max_DNR_per_day,max_GHI_per_day,isday,mean_RH_per_day,mean_GHI_daytime,mean_DNR_daytime,mean_TSC_daytime,mean_WS,mean_WD,mean_DHR_daytime
0,1,1,22.98,75.61,2005-01-01,16.7,31.2,756,663,0,46.875000,192.181818,242.227273,4.136364,2.329167,133.875000,58.818182
1,1,2,22.98,75.61,2005-01-02,14.3,25.8,845,697,0,53.083333,204.818182,293.954545,3.000000,2.162500,154.458333,45.136364
2,1,3,22.98,75.61,2005-01-03,13.3,29.0,1005,774,0,45.791667,224.454545,347.681818,2.727273,1.925000,244.958333,33.363636
3,1,4,22.98,75.61,2005-01-04,12.5,25.4,922,728,0,44.666667,212.954545,320.045455,2.909091,2.016667,186.958333,36.954545
4,1,5,22.98,75.61,2005-01-05,12.7,25.0,677,650,0,40.500000,181.545455,196.454545,3.545455,2.570833,156.375000,75.681818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,12,27,22.98,75.61,2005-12-27,10.6,24.8,838,697,0,57.083333,202.909091,293.681818,2.727273,2.404167,149.500000,43.818182
361,12,28,22.98,75.61,2005-12-28,10.8,25.0,918,726,0,55.416667,207.545455,309.363636,2.227273,1.975000,227.666667,38.363636
362,12,29,22.98,75.61,2005-12-29,13.7,28.0,773,682,0,47.416667,195.909091,261.409091,2.136364,3.508333,200.125000,53.090909
363,12,30,22.98,75.61,2005-12-30,14.9,28.5,773,683,0,47.666667,197.181818,268.545455,2.363636,2.204167,193.708333,52.090909


In [20]:
nndf.columns


Index(['Month', 'Day', 'latitude', 'longitude', 'Days', 'min_DBtemp_per_day',
       'max_DBT_per_day', 'max_DNR_per_day', 'max_GHI_per_day', 'isday',
       'mean_RH_per_day', 'mean_GHI_daytime', 'mean_DNR_daytime',
       'mean_TSC_daytime', 'mean_WS', 'mean_WD', 'mean_DHR_daytime'],
      dtype='object')

In [21]:
class Fuzzify(object):
    def __init__(self, data_series, Min, Max, Mid):
        """Create a triangular membership function with a data series and its max, min and median"""
        
        self.data = data_series
        self.low = Min
        self.high = Max
        self.mid = Mid
        self.universe = np.arange(np.floor(self.low), np.ceil(self.high)+0.2, 0.1)
        self.trimf_low  = fuzz.trimf(self.universe, [self.low, self.low, self.mid])
        self.trimf_mid = fuzz.trimf(self.universe, [self.low, self.mid, self.high+0.1])
        self.trimf_hi  =  fuzz.trimf(self.universe, [self.mid, self.high+0.1, self.high+0.1])
        
    def get_universe(self):
        return self.universe
        
    def get_membership(self, dataset_name):
        """Assign fuzzy membership to each observation in the data series and return a dataframe of the result"""
        
        new_df = pd.DataFrame(self.data)
        new_df[f'low_{dataset_name}'] = fuzz.interp_membership(self.universe, self.trimf_low, self.data)
        new_df[f'mid_{dataset_name}'] = fuzz.interp_membership(self.universe, self.trimf_mid, self.data)
        new_df[f'high_{dataset_name}'] = fuzz.interp_membership(self.universe, self.trimf_hi, self.data)
        new_df[f'membership_{dataset_name}'] = new_df.loc[:, [f'low_{dataset_name}', f'mid_{dataset_name}', f'high_{dataset_name}']].idxmax(axis=1)
        new_df[f'degree_{dataset_name}'] = new_df.loc[:, [f'low_{dataset_name}', f'mid_{dataset_name}', f'high_{dataset_name}']].max(axis=1)
        return new_df


In [22]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd

In [27]:
class myy:
    def getmydata(self,nndf,column_name):
        min_DBtemp_per_day = nndf[column_name]
        
        min_DBtemp_per_day_min = min_DBtemp_per_day.min()
        min_DBtemp_per_day_max =  min_DBtemp_per_day.max()                   
        min_DBtemp_per_day_mid =  np.median(np.arange(min_DBtemp_per_day_min, min_DBtemp_per_day_max, 0.1))            
        
        min_DBtemp_per_day_object = Fuzzify(min_DBtemp_per_day, min_DBtemp_per_day_min, min_DBtemp_per_day_max, min_DBtemp_per_day_mid)
        fuzzified_min_DBtemp_per_day = min_DBtemp_per_day_object.get_membership(column_name)
        
        fuzzified_min_DBtemp_per_day
        nndf = pd.concat([nndf, fuzzified_min_DBtemp_per_day.iloc[:, 4:6]], axis=1)
        return nndf
objj = myy()



In [28]:
nndf=objj.getmydata(nndf,'max_DBT_per_day')
nndf=objj.getmydata(nndf,'min_DBtemp_per_day')
nndf=objj.getmydata(nndf,'max_DNR_per_day')
nndf=objj.getmydata(nndf,'min_DBtemp_per_day')
nndf=objj.getmydata(nndf,'min_DBtemp_per_day')

,Month,Day,latitude,longitude,Days,min_DBtemp_per_day,max_DBT_per_day,max_DNR_per_day,max_GHI_per_day,isday,...,mean_WD,mean_DHR_daytime,membership_max_DBT_per_day,degree_max_DBT_per_day,membership_max_DBT_per_day,degree_max_DBT_per_day,membership_max_DBT_per_day,degree_max_DBT_per_day,membership_min_DBtemp_per_day,degree_min_DBtemp_per_day
0,1,1,22.98,75.61,2005-01-01,16.7,31.2,756,663,0,...,133.875000,58.818182,mid_max_DBT_per_day,0.966387,mid_max_DBT_per_day,0.966387,mid_max_DBT_per_day,0.966387,mid_min_DBtemp_per_day,0.756522
1,1,2,22.98,75.61,2005-01-02,14.3,25.8,845,697,0,...,154.458333,45.136364,mid_max_DBT_per_day,0.572650,mid_max_DBT_per_day,0.572650,mid_max_DBT_per_day,0.572650,mid_min_DBtemp_per_day,0.547826
2,1,3,22.98,75.61,2005-01-03,13.3,29.0,1005,774,0,...,244.958333,33.363636,mid_max_DBT_per_day,0.846154,mid_max_DBT_per_day,0.846154,mid_max_DBT_per_day,0.846154,low_min_DBtemp_per_day,0.539130
3,1,4,22.98,75.61,2005-01-04,12.5,25.4,922,728,0,...,186.958333,36.954545,mid_max_DBT_per_day,0.538462,mid_max_DBT_per_day,0.538462,mid_max_DBT_per_day,0.538462,low_min_DBtemp_per_day,0.608696
4,1,5,22.98,75.61,2005-01-05,12.7,25.0,677,650,0,...,156.375000,75.681818,mid_max_DBT_per_day,0.504274,mid_max_DBT_per_day,0.504274,mid_max_DBT_per_day,0.504274,low_min_DBtemp_per_day,0.591304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,12,27,22.98,75.61,2005-12-27,10.6,24.8,838,697,0,...,149.500000,43.818182,low_max_DBT_per_day,0.512821,low_max_DBT_per_day,0.512821,low_max_DBT_per_day,0.512821,low_min_DBtemp_per_day,0.773913
361,12,28,22.98,75.61,2005-12-28,10.8,25.0,918,726,0,...,227.666667,38.363636,mid_max_DBT_per_day,0.504274,mid_max_DBT_per_day,0.504274,mid_max_DBT_per_day,0.504274,low_min_DBtemp_per_day,0.756522
362,12,29,22.98,75.61,2005-12-29,13.7,28.0,773,682,0,...,200.125000,53.090909,mid_max_DBT_per_day,0.760684,mid_max_DBT_per_day,0.760684,mid_max_DBT_per_day,0.760684,low_min_DBtemp_per_day,0.504348
363,12,30,22.98,75.61,2005-12-30,14.9,28.5,773,683,0,...,193.708333,52.090909,mid_max_DBT_per_day,0.803419,mid_max_DBT_per_day,0.803419,mid_max_DBT_per_day,0.803419,mid_min_DBtemp_per_day,0.600000


In [35]:
nndf = pd.concat([nndf, fuzzified_min_DBtemp_per_day.iloc[:, 4:6]], axis=1)

In [36]:
nndf.columns

Index(['Month', 'Day', 'latitude', 'longitude', 'Days', 'min_DBtemp_per_day',
       'max_DBT_per_day', 'max_DNR_per_day', 'max_GHI_per_day', 'isday',
       'mean_RH_per_day', 'mean_GHI_daytime', 'mean_DNR_daytime',
       'mean_TSC_daytime', 'mean_WS', 'mean_WD', 'mean_DHR_daytime',
       'membership_min_DBtemp_per_day', 'membership_min_DBtemp_per_day',
       'degree_min_DBtemp_per_day'],
      dtype='object')

In [ ]:
class myy:
    def getmydata(self,nndf,column_name):
        min_DBtemp_per_day = nndf[column_name]
        
        min_DBtemp_per_day_min = min_DBtemp_per_day.min()
        min_DBtemp_per_day_max =  min_DBtemp_per_day.max()                   
        min_DBtemp_per_day_mid =  np.median(np.arange(min_DBtemp_per_day_min, min_DBtemp_per_day_max, 0.1))            
        
        min_DBtemp_per_day_object = Fuzzify(min_DBtemp_per_day, min_DBtemp_per_day_min, min_DBtemp_per_day_max, min_DBtemp_per_day_mid)
        fuzzified_min_DBtemp_per_day = min_DBtemp_per_day_object.get_membership(column_name)
        
        fuzzified_min_DBtemp_per_day
        nndf = pd.concat([nndf, fuzzified_min_DBtemp_per_day.iloc[:, 4:6]], axis=1)
        return nndf
objj = myy()

